<a href="https://colab.research.google.com/github/sjeom6/Colab_Repo/blob/main/langchain_240725.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LangChain OpenAI 테스트

In [ ]:
!pip install openai
# !pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.4 MB/s eta 0:00:00


In [ ]:
import os
import openai

os.environ["OPENAI_API_KEY"] = ""

client = openai.OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [ ]:
# 파일 업로드
file = client.files.create(
    file = open("./광주광역시_광주맛집_20220111.csv", "rb"),
    purpose = "assistants"
)


In [ ]:
# code_interpreter 사용
assistant = client.beta.assistants.create(
    model="gpt-4o",
    name = "광주맛집 전문가",
    instructions="사용자가 맛집 정보를 요청할 때, 관련 정보를 데이터베이스에서 정보를 검색하고, 연락처 정보를 알려주세요.",
    tools=[{"type":"code_interpreter"}],
    tool_resources={
      "code_interpreter": {
        "file_ids": [file.id]
      }
    }
  )


In [ ]:
# 스레드 생성
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_GtUOtaInBrSqaBISumbspSai', created_at=1721284612, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [ ]:
# 메시지 생성
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="광주 맛집 정보를 3곳만 알려줘"
)
print(message)

Message(id='msg_334kIpf6Oh7CnOxM1oYCjcr6', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='광주 맛집 정보를 3곳만 알려줘'), type='text')], created_at=1721284831, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_GtUOtaInBrSqaBISumbspSai')


In [ ]:
run = client.beta.threads.runs.create(
      thread_id=thread.id,
      assistant_id=assistant.id,
      # model="gpt-4o",
      tools=[{"type":"code_interpreter"}, {"type":"file_search"}]
    )
print(run)

Run(id='run_nCosyvmxJwarof0wNX6Ei4dA', assistant_id='asst_GSM3n4zKCA62Ve4k2sHXGLTZ', cancelled_at=None, completed_at=None, created_at=1721284859, expires_at=1721285459, failed_at=None, incomplete_details=None, instructions='사용자가 맛집 정보를 요청할 때, 관련 정보를 데이터베이스에서 정보를 검색하고, 연락처 정보를 알려주세요.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_GtUOtaInBrSqaBISumbspSai', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter'), FileSearchTool(type='file_search', file_search=None)], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})


In [ ]:
# 처리결과 확인m
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    run_id=run.id
)

print(messages)


SyncCursorPage[Message](data=[Message(id='msg_FYm32XYLxGoGbKSJoARV8Ysa', assistant_id='asst_GSM3n4zKCA62Ve4k2sHXGLTZ', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='다음은 광주의 맛집 중 세 곳의 정보입니다:\n\n1. **맘스쿡**\n   - 주소: 광주광역시 동구 동계천로143, 1층(동명동)\n   - 전화번호: 062-233-1233\n   - 분류: 향토맛집\n\n2. **옛날밥상**\n   - 주소: 광주광역시 남구 용대로119번길 6(봉선동, 1층)\n   - 전화번호: 062-653-7337\n   - 분류: 향토맛집\n\n3. **송정떡갈비**\n   - 주소: 광주광역시 광산구 광산로29번길 1, 1층(송정동)\n   - 전화번호: 062-944-1439\n   - 분류: 향토맛집\n\n더 필요한 정보나 추가적인 질문이 있으시면 언제든지 말씀해 주세요.'), type='text')], created_at=1721284860, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_nCosyvmxJwarof0wNX6Ei4dA', status=None, thread_id='thread_GtUOtaInBrSqaBISumbspSai')], object='list', first_id='msg_FYm32XYLxGoGbKSJoARV8Ysa', last_id='msg_FYm32XYLxGoGbKSJoARV8Ysa', has_more=False)


In [ ]:
for message in messages:
  if (message.role == "assistant"):
    print(f"어시스턴트: \n{message.content[0].text.value}")
  elif (message.role == "user"):
    print(f"사용자: \n{message.content[0].text.value}")
  else:
    print(f"오류: \n{message.content[0].text.value}")

어시스턴트: 
다음은 광주의 맛집 중 세 곳의 정보입니다:

1. **맘스쿡**
   - 주소: 광주광역시 동구 동계천로143, 1층(동명동)
   - 전화번호: 062-233-1233
   - 분류: 향토맛집

2. **옛날밥상**
   - 주소: 광주광역시 남구 용대로119번길 6(봉선동, 1층)
   - 전화번호: 062-653-7337
   - 분류: 향토맛집

3. **송정떡갈비**
   - 주소: 광주광역시 광산구 광산로29번길 1, 1층(송정동)
   - 전화번호: 062-944-1439
   - 분류: 향토맛집

더 필요한 정보나 추가적인 질문이 있으시면 언제든지 말씀해 주세요.


In [ ]:
# 메시지 생성 2
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="향토맛집도 좋은데 분류가 다양했으면 좋겠어"
)
print(message)

Message(id='msg_8rnsJo2v3oPf4ZKHET5FRAx3', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='향토맛집도 좋은데 분류가 다양했으면 좋겠어'), type='text')], created_at=1721284954, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_GtUOtaInBrSqaBISumbspSai')


In [ ]:
run = client.beta.threads.runs.create(
      thread_id=thread.id,
      assistant_id=assistant.id,
      # model="gpt-4o",
      tools=[{"type":"code_interpreter"}, {"type":"file_search"}]
    )
print(run)

Run(id='run_aS22c2h5dprK1yxuXsdXCRUI', assistant_id='asst_GSM3n4zKCA62Ve4k2sHXGLTZ', cancelled_at=None, completed_at=None, created_at=1721284971, expires_at=1721285571, failed_at=None, incomplete_details=None, instructions='사용자가 맛집 정보를 요청할 때, 관련 정보를 데이터베이스에서 정보를 검색하고, 연락처 정보를 알려주세요.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_GtUOtaInBrSqaBISumbspSai', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter'), FileSearchTool(type='file_search', file_search=None)], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})


In [ ]:
# 처리결과 확인m
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    run_id=run.id
)

print(messages)

SyncCursorPage[Message](data=[Message(id='msg_TCtpXYWIrNBMK5Wq9GaZ6RO5', assistant_id='asst_GSM3n4zKCA62Ve4k2sHXGLTZ', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='다양한 분류의 광주 맛집 세 곳을 추천드립니다:\n\n1. **맘스쿡**\n   - 분류: 향토맛집\n   - 주소: 광주광역시 동구 동계천로143, 1층(동명동)\n   - 전화번호: 062-233-1233\n\n2. **조선옥**\n   - 분류: 한상맛집\n   - 주소: 광주광역시 남구 효덕로 103, 1층(노대동)\n   - 전화번호: 062-654-3322\n\n3. **관가**\n   - 분류: 오래맛집\n   - 주소: 광주광역시 동구 의재로96번길 18(소태동)\n   - 전화번호: 062-226-0040\n\n각 맛집의 분류가 다르므로 취향에 맞게 선택하실 수 있을 것입니다. 다른 정보나 추가 질문이 있으시면 언제든지 말씀해 주세요!'), type='text')], created_at=1721284975, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_aS22c2h5dprK1yxuXsdXCRUI', status=None, thread_id='thread_GtUOtaInBrSqaBISumbspSai')], object='list', first_id='msg_TCtpXYWIrNBMK5Wq9GaZ6RO5', last_id='msg_TCtpXYWIrNBMK5Wq9GaZ6RO5', has_more=False)


In [ ]:
for message in messages:
  if (message.role == "assistant"):
    print(f"어시스턴트: \n{message.content[0].text.value}")
  elif (message.role == "user"):
    print(f"사용자: \n{message.content[0].text.value}")
  else:
    print(f"오류: \n{message.content[0].text.value}")

어시스턴트: 
다양한 분류의 광주 맛집 세 곳을 추천드립니다:

1. **맘스쿡**
   - 분류: 향토맛집
   - 주소: 광주광역시 동구 동계천로143, 1층(동명동)
   - 전화번호: 062-233-1233

2. **조선옥**
   - 분류: 한상맛집
   - 주소: 광주광역시 남구 효덕로 103, 1층(노대동)
   - 전화번호: 062-654-3322

3. **관가**
   - 분류: 오래맛집
   - 주소: 광주광역시 동구 의재로96번길 18(소태동)
   - 전화번호: 062-226-0040

각 맛집의 분류가 다르므로 취향에 맞게 선택하실 수 있을 것입니다. 다른 정보나 추가 질문이 있으시면 언제든지 말씀해 주세요!
